In [ ]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda/lib64'
!pip install pycuda

# Write CUDA C code to a file
cuda_code = """
#include <stdio.h>
#include <curand_kernel.h>
#include <math.h>
#include <stdlib.h>
#include <time.h>

#define N_PARTICLES 1000
#define W 0.8f
#define C1 0.1f
#define C2 0.1f
#define N_ITERATIONS 500
#define BLOCK_SIZE 256

typedef struct {
    float x_pos;
    float y_pos;
    float x_velo;
    float y_velo;
    float x_best;
    float y_best;
    float best;
} particle;

_global_ void pso_kernel(particle *particles, double *gbest, double *gbest_obj, unsigned long long seed) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    curandState state;
    curand_init(seed, index, 0, &state);

    if (index < N_PARTICLES) {
        particle *curr_particle = &particles[index];

        // Initialization
        float x_rand = curand_uniform(&state) * 5;
        float y_rand = curand_uniform(&state) * 5;
        float velo_rand = curand_uniform(&state) * 0.1f;
        curr_particle->x_pos = x_rand;
        curr_particle->y_pos = y_rand;
        curr_particle->x_velo = velo_rand;
        curr_particle->y_velo = velo_rand;
        curr_particle->x_best = x_rand;
        curr_particle->y_best = y_rand;
        curr_particle->best = powf(x_rand - 3.14f, 2) + powf(y_rand - 2.72f, 2) + sinf(3 * x_rand + 1.41f) + sinf(4 * y_rand - 1.73f);

        for (int i = 0; i < N_ITERATIONS; i++) {
            float r1 = curand_uniform(&state);
            float r2 = curand_uniform(&state);

            // Update velocity
            float x_best_diff = curr_particle->x_best - curr_particle->x_pos;
            float y_best_diff = curr_particle->y_best - curr_particle->y_pos;
            float gbest_x_diff = gbest[0] - curr_particle->x_pos;
            float gbest_y_diff = gbest[1] - curr_particle->y_pos;
            curr_particle->x_velo = W * curr_particle->x_velo + C1 * r1 * x_best_diff + C2 * r2 * gbest_x_diff;
            curr_particle->y_velo = W * curr_particle->y_velo + C1 * r1 * y_best_diff + C2 * r2 * gbest_y_diff;

            // Update position
            curr_particle->x_pos += curr_particle->x_velo;
            curr_particle->y_pos += curr_particle->y_velo;

            // Evaluate objective function
            float obj = powf(curr_particle->x_pos - 3.14f, 2) + powf(curr_particle->y_pos - 2.72f, 2) + sinf(3 * curr_particle->x_pos + 1.41f) + sinf(4 * curr_particle->y_pos - 1.73f);
            
            // Update personal best
            if (curr_particle->best > obj) {
                curr_particle->x_best = curr_particle->x_pos;
                curr_particle->y_best = curr_particle->y_pos;
                curr_particle->best = obj;
            }

            __syncthreads();

            // Update global best
            if (curr_particle->best < *gbest_obj) {
                atomicExch((unsigned long long int*)gbest_obj, __double_as_longlong(curr_particle->best));
                atomicExch((unsigned long long int*)gbest, __double_as_longlong(curr_particle->x_pos));
                atomicExch((unsigned long long int*)(gbest + 1), __double_as_longlong(curr_particle->y_pos));
            }
        }
    }
}

int main() {
    particle *d_particles;
    double *d_gbest, *d_gbest_obj;
    double h_gbest[2];
    double h_gbest_obj;

    // Allocate device memory
    cudaMalloc(&d_particles, N_PARTICLES * sizeof(particle));
    cudaMalloc(&d_gbest, 2 * sizeof(double));
    cudaMalloc(&d_gbest_obj, sizeof(double));

    // Generate random seed
    unsigned long long seed = time(NULL);
    clock_t start = clock();
    // Launch kernel
    pso_kernel<<<(N_PARTICLES + BLOCK_SIZE - 1) / BLOCK_SIZE, BLOCK_SIZE>>>(d_particles, d_gbest, d_gbest_obj, seed);
    cudaDeviceSynchronize();
    
    // Copy results back to host
    cudaMemcpy(&h_gbest_obj, d_gbest_obj, sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_gbest, d_gbest, 2 * sizeof(double), cudaMemcpyDeviceToHost);
    clock_t end = clock();
    double total_time = double)(end - start) / CLOCKS_PER_SEC;
    printf("PSO found best solution at f(%lf,%lf)=%lf       total time: %f", h_gbest[0], h_gbest[1], h_gbest_obj, total_time);

    // Free device memory
    cudaFree(d_particles);
    cudaFree(d_gbest);
    cudaFree(d_gbest_obj);

    return 0;
}


"""
with open("cuda1.cu", "w") as f:
    f.write(cuda_code)

# Compile CUDA code
!nvcc -o h cuda1.cu

# Run compiled binary
!./h